In [20]:
from bokeh.io import curdoc, output_notebook
from bokeh.layouts import layout, widgetbox, column
from bokeh.models import Div, LinearColorMapper, LogColorMapper

from bokeh.models import ContinuousColorMapper

from bokeh.models import ColumnDataSource, GeoJSONDataSource, HoverTool, DateRangeSlider
from bokeh.models.widgets import Select
from bokeh.plotting import figure, show

from bokeh.models import ColumnDataSource

import utils.mapper_utils as mapper_utils
import pandas as pd

import geopandas as gpd

from colour import Color

import pickle


In [2]:
import bokeh
[ i for i in bokeh.models.__dict__.keys() if 'ColorMapper' in i ]


['ContinuousColorMapper',
 'ColorMapper',
 'CategoricalColorMapper',
 'LogColorMapper',
 'LinearColorMapper']

In [3]:
min_long = -87.52
min_lat = 41.625
max_long = -87.875
max_lat = 42.05

shapefile_aliases = ["Wards (2003-2015)", "Wards (2015-Present)", "Neighborhoods"]

In [4]:
blah = [1,2,3,4,5,6]

In [5]:
output_notebook()

Loading BokehJS ...

In [6]:
def create_map_figure():
    fig = figure(plot_height=800, plot_width=800, title="Tickets!",
                           x_range=(max_long, min_long),
                           y_range=(min_lat, max_lat),
                           background_fill_color=None)

    fig.toolbar.active_scroll = fig.tools[1]

    fig.xgrid.grid_line_color = None
    fig.ygrid.grid_line_color = None

    return fig

In [7]:
def get_boundaries(selected_shapefile=None, counts={}):
    filepath, data_source = None, None

    boundaries = []
    if selected_shapefile == "Wards (2003-2015)":
        filedir = '/opt/data/shapefiles/wards2003'
        filepath = '%s/geo_export_40f24d1b-c326-4d93-ad6f-33c8a1b35fe6.shp' % filedir
        geolines = gpd.read_file(filepath)


    elif selected_shapefile == "Wards (2015-Present)":
        filedir = '/opt/data/shapefiles/wards2015'
        filepath = '%s/geo_export_21178b31-9758-4895-adcc-24ce80929959.shp' % filedir
        geolines = gpd.read_file(filepath)


    elif selected_shapefile == "Neighborhoods":
        filedir = '/opt/data/shapefiles/neighborhoods/'
        filepath = '%s/geo_export_11129122-2d69-48a4-9b49-ae802351855f.shp' % filedir
        geolines = gpd.read_file(filepath)

    elif selected_shapefile == "Census":
        filedir = '/opt/data/shapefiles/neighborhoods/'
        filepath = '%s/geo_export_11129122-2d69-48a4-9b49-ae802351855f.shp' % filedir
        geolines = gpd.read_file(filepath)
        
    fill_counts = []
    for community in geolines['community']:
        community_count = 0
        for count_place, count in counts.items():
            if community.title() == count_place:
                community_count = count
        fill_counts.append(community_count)

    if fill_counts:
        print(fill_counts)
        geolines['ticket_counts'] = fill_counts

    data_source = GeoJSONDataSource(geojson=geolines.to_json())
        

    return data_source

In [8]:
blah = {'123':1234}
blah.items()

dict_items([('123', 1234)])

In [9]:
filedir = '/opt/data/shapefiles/neighborhoods/'
filepath = '%s/geo_export_11129122-2d69-48a4-9b49-ae802351855f.shp' % filedir

if filepath:
    geolines = gpd.read_file(filepath)

In [12]:
#try:
fh = open('/opt/data/tickets/parking/tickets_source.pkl','rb')
tickets_source = pickle.load(fh)
#except:
#    print(":(")
#    tickets_source = mapper_utils.get_tickets(19000000)

tickets_cache = tickets_source.copy()
#ticket_points = map_figure.circle(x='longitude', y='latitude', source=tickets_source, color="black", size=.5)

map_figure = create_map_figure()

wards2003_counts = tickets_source.ward2003_no.value_counts()
wards2015_counts = tickets_source.ward2015_no.value_counts()
neighborhood_counts = tickets_source.neighborhood.value_counts()

from bokeh.palettes import RdBu11 as palette
color_mapper = LogColorMapper(palette=palette)
fill_color={'field': 'ticket_counts', 'transform': color_mapper}

#data_source = get_boundaries("Neighborhoods")
neighborhoods_lines_data = get_boundaries("Neighborhoods", counts=neighborhood_counts)
neighborhood_lines = map_figure.patches('xs', 'ys', source=neighborhoods_lines_data, 
                                        name="Neighborhoods", visible=False, fill_color=fill_color, fill_alpha=.8)


[69498, 14163, 11966, 117895, 66733, 335144, 68899, 247831, 128790, 428143, 64093, 23069, 63983, 393057, 280334, 302347, 97682, 32637, 385489, 467127, 147632, 283313, 678379, 339561, 1147224, 613402, 148640, 98274, 611093, 187561, 473750, 587419, 316989, 157824, 118230, 45847, 1111651, 659755, 350980, 254237, 39569, 159267, 8477, 61653, 205693, 221545, 23059, 34150, 59517, 80365, 31582, 19774, 194963, 71690, 259641, 60098, 1580245, 142476, 257971, 67996, 186215, 64337, 112968, 325581, 200832, 152801, 232946, 954844, 77690, 253673, 52128, 110763, 21263, 65303, 224754, 427502, 18243]


In [ ]:
#fh = open('/opt/data/tickets/parking/tickets_source.pkl','wb')
#pickle.dump(tickets_source, fh, pickle.HIGHEST_PROTOCOL)

In [ ]:
#ward2003_lines_data = get_boundaries("Wards (2003-2015)")
#wardpresent_lines_data = get_boundaries("Wards (2015-Present)")

#ward2003_lines = map_figure.patches('xs', 'ys', source=ward2003_lines_data, name="Wards (2003-2015)", visible=False)
#wardpresent_lines = map_figure.patches('xs', 'ys', source=wardpresent_lines_data, name="Wards (2015-Present)", visible=False)

In [13]:
def lines_update():
    for lines in [neighborhood_lines]: #, ward2003_lines, wardpresent_lines]:
        lines.visible = False
        
    val = lines_selecter.value
    
    if val == "Wards (2003-2015)":
        ward2003_lines.visible = True
    elif val == "Neighborhoods (2015-Present)":
        wardpresent_lines.visible = True        
    else:
        neighborhood_lines.visible = True

In [14]:
options = ["Neighborhoods", "Wards (2003-2015)", "Wards (2015-Present)"]
lines_selecter = Select(title="Boundaries", value=options[0], options=options)
lines_selecter.on_change('value', lambda attr, old, new: lines_update())

In [15]:
ghover = HoverTool(renderers=[neighborhood_lines])
ghover.tooltips=[("community", "@community")]
map_figure.add_tools(ghover)

In [16]:
viol_opts_counts = tickets_source.violation_code.value_counts()
viol_options = ["%s (%s)" % (i, viol_opts_counts[i]) for i in viol_opts_counts.index]    

violations_selecter = Select(title="Violation", value="0964190A (142)", options=viol_options) 

In [17]:
def update_tickets():
    tickets_source = tickets_cache[(issue_date > daterange_slider.value)]

In [18]:
min_range = tickets_source.issue_date.min()
max_range = tickets_source.issue_date.max()

daterange_slider = DateRangeSlider(title="Date", start=min_range, end=max_range, value=(min_range, max_range))
daterange_slider.on_change('value', lambda attr, old, new: tickets_update())
#pd.to_datetime()

In [19]:
#data_source = get_boundaries("Wards (2003-2015)")
lines_update()
col = column([daterange_slider, violations_selecter, lines_selecter, map_figure])

show(col)

#### tickets_doc = curdoc()

selecter = Select(title="Boundaries", value="Neighborhoods", options=shapefile_aliases)
controls = [selecter]


for control in controls:
    control.on_change('value', lambda attr, old, new: app.update())

desc = column()
inputs = widgetbox(*mapped_controls, sizing_mode='fixed')

bokeh_objs = [inputs, boundary_fig, mapfigure]
layout = layout([[desc], bokeh_objs], sizing_mode='fixed')

tickets_doc.add_root(layout)
tickets_doc.title = "Parking Tickets!"